# Part 1

In [1]:
import numpy as np
import hashlib as hs

## Importing Data

Import the unsalted and unstreched password file that contains usernames and hashes of users that was retrieved by the victims server.  
Import the most used password database called "rockyou.txt" to be used in each part of the project.

In [2]:
passwords = np.genfromtxt("digitalcorp.txt", delimiter = ',', dtype = str, skip_header = 1)
rock_you = np.genfromtxt("rockyou.txt", dtype = str)

## Exporting Attack Table

Using SHA-512 algorithm to encrypt each password that is in the rockyou database and export the resulting table in (password,hash) pairs to a .csv file so we can use it without calculating the values each time.

In [3]:
attack_table = []
for password in rock_you:
    sha512 = hs.sha512()
    sha512.update(password.encode("ASCII"))
    attack_table.append([password, sha512.hexdigest()])

np.savetxt("attack_table.csv", attack_table, fmt = "%s,%s")

## Importing the Attack Table

Import the generated rainbow table we calculated earlier and compare each hash with the hashed we obtained from the victims to search for a match.

In [4]:
attack = np.genfromtxt("attack_table.csv", delimiter = ',', dtype = str)
for user,hashed in passwords:
    for password, hashed_pass in attack:
        if hashed_pass == hashed:
            print("Password of " + user + " is: " + password)

Password of Creed is: cocacola
Password of Meredith is: 50cent
Password of Stanley is: patrick
Password of Phyllis is: newyork


## Results

Password of Creed is: cocacola  
Password of Meredith is: 50cent  
Password of Stanley is: patrick  
Password of Phyllis is: newyork  

# Part 2

## Importing Data

Import the file that contains usernames, salts and hashes of users from the victims servers.

In [5]:
passwords = np.genfromtxt("salty-digitalcorp.txt", delimiter = ',', dtype = str, skip_header = 1)

## Exporting Attack Table

When using salts most simple method is appending at the front of the password or at the end. So, we calculate two hashes for each user and export the rainbow table so we can use it later.

In [6]:
attack_table = []
for password in rock_you:
    for _,salt,_ in passwords:
        salted = salt + password
        sha512 = hs.sha512()
        sha512.update(salted.encode("ASCII"))
        hash_1 = sha512.hexdigest()
        salted = password + salt
        sha512 = hs.sha512()
        sha512.update(salted.encode("ASCII"))
        hash_2 = sha512.hexdigest()
        attack_table.append([password, hash_1, hash_2])
np.savetxt("attack_table_salted.csv", attack_table, fmt = "%s,%s,%s")

## Importing Attack Table

We import the rainbow table that we calculated earlier and compare the hashes with the hashes we obtained from the victims to find a match.

In [7]:
attack = np.genfromtxt("attack_table_salted.csv", delimiter = ',', dtype = str)
for user,_,hashed in passwords:
    for password, hash_1, hash_2 in attack:
        if hash_1 == hashed or hash_2 == hashed:
            print("Password of " + user + " is: " + password)

Password of Kevin is: tinkerbell
Password of Angela is: chrisbrown
Password of Oscar is: chivas
Password of Darryl is: eminem


## Results

Password of Kevin is: tinkerbell  
Password of Angela is: chrisbrown  
Password of Oscar is: chivas  
Password of Darryl is: eminem  

# Part 3

## Importing Data

Import the file that was key streched containing username, salt and hashes of the victims that was obtained from the victims server. 

In [8]:
passwords = np.genfromtxt("keystreching-digitalcorp.txt", delimiter = ',', dtype = str, skip_header = 1)

In [9]:
#This is a helper method to print the found patter that was used in key streching.
def print_pattern(index):
    if index == 0:
        print("Pattern is [hash + password + salt]")
    if index == 1:
        print("Pattern is [hash + salt + password]")
    if index == 2:
        print("Pattern is [password + salt + hash]")
    if index == 3:
        print("Pattern is [password + hash + salt]")
    if index == 4:
        print("Pattern is [salt + hash + password]")
    if index == 5:
        print("Pattern is [salt + password + hash]")
    if index == 6:
        print("Pattern is [hash + password + salt]")
    if index == 7:
        print("Pattern is [hash + salt + password]")
    if index == 8:
        print("Pattern is [password + salt + hash]")
    if index == 9:
        print("Pattern is [password + hash + salt]")
    if index == 10:
        print("Pattern is [salt + hash + password]")
    if index == 11:
        print("Pattern is [salt + password + hash]")
        

## Running the Attack

We have to calculate all possible patterns and number of times key streching is used. This results in millions of possible hash combinations so we don't export the rainbow table calculated here because its size results in gigabytes instead we keep it in memory and print the pattern that was used and number of iterations it was key streched.  
We know that simple method of arranging is used to calculate the next hash so with 3 items and initial hashing has 2 possible combinations (similar to part 2). For each known password in rockyou database we have to calculate 12 possible combinations of ordering for each user. We have 4 users in total and we know that number of iterations is between 1-2000. So, we have to calculate 502x4x12x2000 = 48,192,000 possible hashes, that's why we don't export the table.  
First, we go through each password in rockyou database one-by-one and calculate 12 possible initial salting (first 6 password + salt, last 6 salt + password), then we calculate the next hash for each 12 possible combinations of hash, salt and password. We then update the 12 hashes we calculated and compare the each one with the hash of the victims. When a match is found we print the number of iterations that was key streched and the patter that was used. 

In [ ]:
skip = False
for password in rock_you:
    for user,salt,hashed in passwords:
        skip = False
        hash_functions = [hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),
                          hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512()]
        hashes = []
        initial_hash_1 = password + salt
        initial_hash_2 = salt + password
        for i in range(len(hash_functions)):
            if i < len(hash_functions)/2:
                hash_functions[i].update(initial_hash_1.encode("ASCII"))
                hashes.append(hash_functions[i].hexdigest())
            else:
                hash_functions[i].update(initial_hash_2.encode("ASCII"))
                hashes.append(hash_functions[i].hexdigest())
        for i in range(2000):
            hash_functions = [hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),
                              hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512(),hs.sha512()]
            hash_functions[0].update(str(hashes[0] + password + salt).encode("ASCII"))
            hash_functions[1].update(str(hashes[1] + salt + password).encode("ASCII"))
            hash_functions[2].update(str(password + salt + hashes[2]).encode("ASCII"))
            hash_functions[3].update(str(password + hashes[3] + salt).encode("ASCII"))
            hash_functions[4].update(str(salt + hashes[4] + password).encode("ASCII"))
            hash_functions[5].update(str(salt + password + hashes[5]).encode("ASCII"))
            hash_functions[6].update(str(hashes[6] + password + salt).encode("ASCII"))
            hash_functions[7].update(str(hashes[7] + salt + password).encode("ASCII"))
            hash_functions[8].update(str(password + salt + hashes[8]).encode("ASCII"))
            hash_functions[9].update(str(password + hashes[9] + salt).encode("ASCII"))
            hash_functions[10].update(str(salt + hashes[10] + password).encode("ASCII"))
            hash_functions[11].update(str(salt + password + hashes[11]).encode("ASCII"))
            for j in range(len(hash_functions)):
                hashes[j] = hash_functions[j].hexdigest()
                if hashes[j] == hashed:
                    print("Password of " + user + " is: " + password)
                    print("Key streched with ", i + 1, " iterations.")
                    print_pattern(j)
                    skip = True
                    break
            if skip:
                break
            


Password of Jim is: hottie
Key streched with  1258  iterations.
Pattern is [password + hash + salt]
Password of Dwight is: angelica
Key streched with  1258  iterations.
Pattern is [password + hash + salt]
Password of Pam is: cutiepie
Key streched with  1258  iterations.
Pattern is [password + hash + salt]
Password of Michael is: superstar
Key streched with  1258  iterations.
Pattern is [password + hash + salt]


## Results

Number of iterations that was key streched: 1258  
Pattern that was used for the next hash: password + previous_hash + salt  
Password of Jim is: hottie  
Password of Dwight is: angelica  
Password of Pam is: cutiepie  
Password of Michael is: superstar